In [1]:
# 필요 모듈 호출
import tensorflow as tf
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. 데이터 수집하기

In [2]:
'''
mkdir -p ~/aiffel/transformer_chatbot/data/
ln -s ~/data/* ~/aiffel/transformer_chatbot/data/
'''
conversation = pd.read_csv(os.getenv('HOME')+'/aiffel/transformer_chatbot/data/ChatbotData .csv')
conversation.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
MAX_SAMPLES = len(conversation)
MAX_SAMPLES

11823

In [4]:
conversation.tail()

,Q,A,label
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.,2


## 2. 데이터 전처리하기

In [5]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.strip()
    
    # 단어와 구두점 사이 거리
    sentence = re.sub(r'([?.,!])',r' \1 ', sentence)
    sentence = re.sub(r'[" "]+', ' ', sentence)
    
    return sentence

In [6]:
conversation['Q'][1]

'1지망 학교 떨어졌어'

In [7]:
# 질문과 답변의 쌓인 데이터셋 구성 위한 데이터 로드 함수
def load_conversations():
    inputs, outputs = [], []
    for i in range(len(conversation)):
        inputs.append(preprocess_sentence(conversation['Q'][i]))
        outputs.append(preprocess_sentence(conversation['A'][i]))
    
    return inputs, outputs

In [8]:
# 데이터를 로드하고 전처리하여 질문과 답변을 저장
questions, answers = load_conversations()

print('전체 샘플 수:', len(questions))
print('전체 샘플 수:', len(questions))

전체 샘플 수: 11823
전체 샘플 수: 11823


In [9]:
print('전처리 후의 22번째 질문 샘플: {}'.format(questions[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[21]))

전처리 후의 22번째 질문 샘플: 가스비 장난 아님
전처리 후의 22번째 답변 샘플: 다음 달에는 더 절약해봐요 . 


## 3. SubwordTextEncoder 사용하기

In [10]:
# 1. 단어장(vocabulary) 만들기
import tensorflow_datasets as tfds

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [11]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여한다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [12]:
print('START_TOKEN의 번호: ', [tokenizer.vocab_size])
print('END_TOKEN의 번호: ', [tokenizer.vocab_size + 1])

START_TOKEN의 번호:  [8170]
END_TOKEN의 번호:  [8171]


In [13]:
# 시작 토큰과 종료 토큰을 고려해 +2를 하여 단어장의 크기를 산정한다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8172


In [14]:
# 2. 각 단어 정수 인코딩 & 패딩
# 임의의 22번째 샘플에 대해 정수 인코딩 작업을 수행
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5761, 609, 2489, 4160]
정수 인코딩 후의 21번째 답변 샘플: [2353, 7510, 7, 6273, 96, 1]


In [15]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 12
print(MAX_LENGTH)

12


In [16]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    
    for (sentence1, sentence2) in zip(inputs, outputs):
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        
        # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
            
    # 최대 길이 40으로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(\
                                                                        tokenized_inputs, 
                                                                        maxlen=MAX_LENGTH, 
                                                                        padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(\
                                                                         tokenized_outputs,
                                                                         maxlen=MAX_LENGTH,
                                                                         padding='post')
        
    return tokenized_inputs, tokenized_outputs

In [17]:
questions, answers = tokenize_and_filter(questions, answers)

print('단어장의 크기: ', VOCAB_SIZE)
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기:  8172
필터링 후의 질문 샘플 개수: 10739
필터링 후의 답변 샘플 개수: 10739


In [18]:
# 3. 교사 강요 사용
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용한다.
# 이에 따라 outputs에서는 START_TOKEN을 제거한다.
dataset = tf.data.Dataset.from_tensor_slices(({
    'inputs' : questions,
    'dec_inputs' : answers[:,:-1]
},{
    'outputs' : answers[:, 1:]
}))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## 4. 모델 구성하기
### 1) 인코더 및 디코더 정의

In [19]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):
    
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)
        
    def get_angles(self, position, i, d_model):
        angles = 1 /tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        # 각도 배열 생성
        angle_rads = self.get_angles(\
                                    position = tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                                    i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                                    d_model = d_model)
        
        # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:,0::2])
        # 배역의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:,1::2])
        
        # sin과 cosine 이 교차되도록 재배열
        pos_encoding = tf.stack([sines, cosines],axis=0)
        pos_encoding = tf.transpose(pos_encoding, [1,2,0])
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])
        
        pos_encoding = pos_encoding[tf.newaxis, ...]
        
        return tf.cast(pos_encoding, tf.float32)
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
    

In [20]:
# scaled dot product attention function
def scaled_dot_product_attention(query, key, value, mask):
    # 어텐션 가중치는 Q와 K의 내적
    matmul_qk = tf.matmul(query, key, transpose_b = True)
    
    # 가중치 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)
    
    # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)
        
    # softmax 적용
    attention_weights = tf.nn.softmax(logits, axis=-1)
    
    # 최종 attention은 가중치와 v의 내적
    output = tf.matmul(attention_weights, value)
    
    return output


In [21]:
# Multi-head attention 
class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0 # assert: 가정
        
        self.depth = d_model // self.num_heads
        
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        
        self.dense = tf.keras.layers.Dense(units=d_model)
        
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs,
                           shape = (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0,2,1,3])
    
    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        
        # Q, K, V 에 각각 Dense를 적용
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        
        # 병렬 연산을 위한 머리를 여러 개 만든다.
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        
        # scaled dot product attention 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm = [0,2,1,3]) # 왜 perm이 이 순서???
        
        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)한다.
        concat_attention = tf.reshape(scaled_attention,
                                     (batch_size, -1, self.d_model))
        
        # 최종 결과에도 Dense를 한 번 더 적용 -> 왜...??
        outputs = self.dense(concat_attention)
        
        return outputs

In [22]:
# 1. Padding Masking
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x,0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [23]:
# 2. Look-ahead masking
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [24]:
# 인코더 하나의 레이어를 함수로 구현
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재한다.
def encoder_layer(units, d_model, num_heads, dropout, name='encoder_layer'):
    inputs = tf.keras.Input(shape=(None, d_model), name='inputs')
    
    # Padding Mask 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
    
    # 첫 번째 서브 레이어 : Multi-Head attention수행 (self attention)
    attention = MultiHeadAttention(d_model, num_heads,
                                  name = 'attention')({'query':inputs,
                                                       'key':inputs,
                                                       'value':inputs,
                                                       'mask':padding_mask
    })
    
    # attention의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉 수행
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)
    
    # 두 번째 서브 레이어 : 2 개의 완전 연결층
    outputs = tf.keras.layers.Dense(units = units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units = d_model)(outputs)
    
    # 완전연결층의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention+outputs) # 왜 더하지..???
    
    return tf.keras.Model(inputs = [inputs, padding_mask], outputs=outputs, name=name)

In [25]:
# 인코더 층을 쌓아 인코더 만들기
def encoder(vocab_size,
           num_layers,
           units,
           d_model,
           num_heads,
           dropout,
           name='encoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    
    # Padding Mask 사용 >> 이렇게만 하면 마스킹이 됨??????
    padding_mask = tf.keras.Input(shape=(1,1, None), name='padding_mask')
    
    # Embedding Layer
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32)) 
    
    # Positional Encoding
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings) # dropout을 왜 여기서.???
    
    # num_layers만큼 쌓아올린 인코더의 층
    for i in range(num_layers):
        outputs = encoder_layer(units = units,
                               d_model = d_model,
                               num_heads=num_heads,
                               dropout=dropout,
                               name='encoder_layer_{}'.format(i),
                               )([outputs, padding_mask])
        
        return tf.keras.Model(inputs=[inputs, padding_mask],
                             outputs = outputs, name=name)

In [26]:
# 디코더 하나의 레이어를 함수로 구현
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재한다.
def decoder_layer(units, d_model, num_heads, dropout, name='decoder_layer'):
    
    inputs = tf.keras.Input(shape=(None, d_model), name='inputs')
    
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    
    padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')
    
    # 첫 번째 서브 레이어 : Multi Head Attention 수행 (self attention)
    attention1 = MultiHeadAttention(d_model, num_heads, 
                                   name='attention_1')(inputs={
        'query': inputs,
        'key': inputs,
        'value': inputs,
        'mask' : look_ahead_mask
    })
    
    # Multi head attention의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1+inputs)
    
    # 두 번째 서브 레이어 : Masked Multi-head attention 수행 (인코더-디코더 attention)
    attention2 = MultiHeadAttention(d_model, num_heads, name='attention_2')(inputs={
        'query':attention1,
        'key':enc_outputs,
        'value':enc_outputs,
        'mask':padding_mask
    })
    
    # Masked Multi-head attention결과는 
    # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉 수행
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2+attention1)
    
    # 세 번째 서브 레이어 : 2개의 완전 연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    
    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)
    
    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
                         outputs = outputs,
                         name=name)

In [27]:
# 디코더 층을 쌓아 디코더 만들기
def decoder(vocab_size,
           num_layers, 
           units,
           d_model,
           num_heads,
           dropout,
           name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    
    # Padding Mask
    padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')
    
    # Embedding layer
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    
    # Positional Encoding
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    
    # Dropout이라는 훈련 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
    
    for i in range(num_layers):
        outputs = decoder_layer(units=units, 
                               d_model = d_model,
                               num_heads=num_heads,
                               dropout=dropout,
                               name='decoder_layer_{}'.format(i),
                               )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])
        
    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
                         outputs = outputs,
                         name=name)

### 2) 트랜스포머 함수 정의

In [28]:
def transformer(vocab_size,
               num_layers,
               units,
               d_model,
               num_heads,
               dropout,
               name='transformer'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    dec_inputs = tf.keras.Input(shape=(None,), name='dec_inputs')
    
    # 인코더에서 패딩을 위한 마스크
    enc_padding_mask = tf.keras.layers.Lambda(\
                                             create_padding_mask,
                                             output_shape=(1,1,None),
                                             name='enc_padding_mask')(inputs)
    
    # 디코더에서 미래의 토큰을 마스크 하기 위해 사용한다.
    # 내부적으로 패딩 마스크도 포함되어 있다.
    look_ahead_mask = tf.keras.layers.Lambda(\
                                            create_look_ahead_mask,
                                            output_shape=(1, None, None),
                                            name='look_ahead_mask')(dec_inputs)
    
    # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(\
                                             create_padding_mask,
                                             output_shape=(1,1,None),
                                             name='dec_padding_mask')(inputs)
    
    # 인코더
    enc_outputs = encoder(\
                         vocab_size=vocab_size,
                         num_layers=num_layers,
                         units=units,
                         d_model=d_model, 
                         num_heads=num_heads,
                         dropout=dropout,
                         )(inputs=[inputs, enc_padding_mask])
    
    # 디코더
    dec_outputs = decoder(\
                         vocab_size=vocab_size,
                         num_layers=num_layers,
                         units=units,
                         d_model=d_model,
                         num_heads=num_heads,
                         dropout=dropout,
                         )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])
    
    # 완전 연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name='outputs')(dec_outputs)
    
    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs = outputs, name=name)

### 3) 모델 생성

In [29]:
tf.keras.backend.clear_session()

# 하이퍼 파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입/출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭 아웃의 비율

model = transformer(\
                   vocab_size = VOCAB_SIZE,
                   num_layers = NUM_LAYERS,
                   units = UNITS,
                   d_model = D_MODEL,
                   num_heads = NUM_HEADS,
                   dropout = DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    2619136     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

### 4) 손실함수

In [30]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    # 왜 reshape하고 왜저렇게 함?
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(\
                                                        from_logits = True,
                                                        reduction='none')(y_true, y_pred)
    
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    
    return tf.reduce_mean(loss)

### 5) 커스텀된 학습률

In [31]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        
        self.warmup_steps = warmup_steps
        
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    

### 6) 모델 compile

In [32]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(\
                                    learning_rate,
                                    beta_1=0.9,
                                    beta_2=0.98,
                                    epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

### 7) 훈련하기

In [33]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/50
168/168 [==============================] - 10s 26ms/step - loss: 4.8575 - accuracy: 0.0963
Epoch 2/50
168/168 [==============================] - 4s 26ms/step - loss: 3.9643 - accuracy: 0.1764
Epoch 3/50
168/168 [==============================] - 5s 27ms/step - loss: 3.3242 - accuracy: 0.1793
Epoch 4/50
168/168 [==============================] - 5s 27ms/step - loss: 3.0437 - accuracy: 0.1889
Epoch 5/50
168/168 [==============================] - 5s 27ms/step - loss: 2.8624 - accuracy: 0.1984
Epoch 6/50
168/168 [==============================] - 5s 27ms/step - loss: 2.6808 - accuracy: 0.2105
Epoch 7/50
168/168 [==============================] - 5s 27ms/step - loss: 2.4809 - accuracy: 0.2278
Epoch 8/50
168/168 [==============================] - 4s 26ms/step - loss: 2.2534 - accuracy: 0.2513
Epoch 9/50
168/168 [==============================] - 5s 27ms/step - loss: 1.9974 - accuracy: 0.2810
Epoch 10/50
168/168 [==============================] - 4s 26ms/step - loss: 1.7246 - accur

## 5. 모델 평가하기

In [34]:
# inference 함수 만들기
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)
    
    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞 뒤로 추가
    sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis = 0)
    
    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장
    output_sequence = tf.expand_dims(START_TOKEN, 0)
    
    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측 반복
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]
        
        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
            
        # 예측한 단어들은 지속적으로 output_sequence에 추가된다.
        # 이 output_sequence는 다시 디코더의 입력이 된다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)
    return tf.squeeze(output_sequence, axis=0)

In [35]:
# 대답 얻는 함수
def sentence_generation(sentence):
    # 입력 문장에 대해 디코더를 동작시켜 예측된 정수 시퀀스를 리턴받는다.
    prediction = decoder_inference(sentence)
    
    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환한다.
    predicted_sentence = tokenizer.decode(\
                                         [i for i in prediction if i < tokenizer.vocab_size])
    
    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))
    
    return predicted_sentence

In [36]:
sentence_generation('요즘 어때?')

입력 : 요즘 어때?
출력 : 괜찮은 사람이에요 . 


'괜찮은 사람이에요 . '

In [37]:
conversation[conversation['A']=='괜찮은 사람이에요.']

,Q,A,label
465,나 괜찮지 않니,괜찮은 사람이에요.,0
517,나 어때?,괜찮은 사람이에요.,0


In [38]:
sentence_generation('어디 갈까')

입력 : 어디 갈까
출력 : 연인과 함께면 어디든 좋아요 . 


'연인과 함께면 어디든 좋아요 . '

In [46]:
conversation[conversation['A']=='연인과 함께면 어디든 좋아요.']

,Q,A,label
10646,여자친구랑 어디 놀러갈까?,연인과 함께면 어디든 좋아요.,2


In [39]:
conversation[conversation['A']=='오늘은 예능이요.']

,Q,A,label
1809,뭐 볼까?,오늘은 예능이요.,0
3295,오늘 뭐 재미있는거 하는 날인가,오늘은 예능이요.,0


In [40]:
sentence_generation('뭐 좋아해')

입력 : 뭐 좋아해
출력 : 고백하세요 . 


'고백하세요 . '

In [41]:
conversation[conversation['A']=='고백하세요.']

,Q,A,label
11279,좋아해,고백하세요.,2


In [42]:
sentence_generation('핸드크림 발라봤니')

입력 : 핸드크림 발라봤니
출력 : 받는 사람은 정말 좋겠어요 . 


'받는 사람은 정말 좋겠어요 . '

In [43]:
conversation[conversation['A']=='공부를 많이 해놨나봐요.']

,Q,A,label
2755,시험 준비 빨리 끝났으면,공부를 많이 해놨나봐요.,0
2756,시험 준비 언제 끝나,공부를 많이 해놨나봐요.,0


In [44]:
sentence_generation('발라봤니')

입력 : 발라봤니
출력 : 잘 찾아보세요 . 


'잘 찾아보세요 . '

In [45]:
conversation[conversation['A']=='이제 보지 마세요.']

,Q,A,label
8356,카톡프사 무슨 의미일까?,이제 보지 마세요.,1


* * *
### 회고
결과는 그렇게 좋진 않다. 하지만 epoch를 높이니 조금씩 나아지는 듯 하다.\
처음 학습 epoch를 20으로 돌렸을 때, 그럴 듯한 답변이 하나도 나오지 않았는데 \
50으로 높이니 몇 개는 그나마 그럴 듯 하다. \
sentence_generation으로 나온 답변이 50으로 돌린 답변이고, 그 아래 dataframe으로 나온 답변이 이전에 나온 답변.

학습데이터의 수도 부족한데 답변과 질문도 한정적이라서 영어에 비해 좋은 성능은 나오지 않은 것 같다. 학습데이터에 있는 답변을 그대로 가져와서 내보낸 것 을 보니 한국어를 형태소로 뜯어보지 않았기 때문일 수도 있다.  물론 나의 한국어 전처리가 많이 부실하기도 했다. 형태소로 뜯어서 적용했으면 결과가 확실히 좋아졌을까?

attention이 어떻게 구성되고 어떤 식으로 흘러가는 지 이해할 수 있는 노드였다. 하지만 인코더와 디코더 포지셔널 인코딩 등 세부적으로 뜯어봤을 때 이해하지 못한 부분이 있어서 추가적으로 계속 공부해야겠다.